# Imports & Initialization

In [1]:
!pip install sypy tqdm dill reportlab # uncomment if the imports don't work
# !sudo apt update
# !sudo apt install texlive-latex-base

In [2]:
import os
import sympy              # for symbolic modelling
from sympy import symbols, Matrix, oo, Eq, simplify, solve, latex, denom, numer, sqrt, collect, degree
from tqdm import tqdm     # to create progress bars
from pprint import pprint # pretty print feature


# optional imports
import dill               # to save/load the results
# from google.colab import files
from multiprocessing import Pool, Value, cpu_count, Manager
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Create the directory if it doesn't exist
print("# of cores: {}".format(os.cpu_count())) # is 96 for TPU v2-8

# # symbols? #use this to find documentation on any object/functionts

# of cores: 8


# Algorithms Definitions

## Set up global variables

In [3]:
# Define symbolic variables
s = symbols('s')
R1, R2, R3, R4, R5, RL, Rs = symbols('R1 R2 R3 R4 R5 RL Rs')
C1, C2, C3, C4, C5, CL = symbols('C1 C2 C3 C4 C5 CL')
L1, L2, L3, L4, L5, LL = symbols('L1 L2 L3 L4 L5 LL')
Z1 , Z2 , Z3 , Z4 , Z5 , ZL, Zs = symbols('Z1 Z2 Z3 Z4 Z5 ZL Zs')

# Transmission matrix coefficients
gm, ro, Cgd, Cgs = symbols('gm ro Cgd Cgs')
a11, a12, a21, a22 = symbols('a11 a12 a21 a22')

T_select ={
"simple" : Matrix([[0, -1/gm],[0, 0]]),
"symbolic" : Matrix([[a11, a12],[a21, a22]]),
"some_parasitic" : Matrix([[-1/(gm*ro), -1/gm],[0, 0]]),
"full_parasitic" : Matrix([[(1/ro + s*Cgd)/(s*Cgd - gm), 1/(s*Cgd - gm)],[(Cgd*Cgs*ro*s + Cgd*gm*ro + Cgs + Cgd)*s/(s*Cgd - gm), (Cgs+Cgd)*s/(s*Cgd - gm)]])
}

# Select a transimision matrix
T_type = "symbolic"
T_a = T_select[T_type]
T_b = T_select[T_type]

print(f"Transmission matrices are type: {T_type}")
print(T_a)
print(T_b)

Transmission matrices are type: symbolic
Matrix([[a11, a12], [a21, a22]])
Matrix([[a11, a12], [a21, a22]])


## Algorithm 1: Solve for TF
Algorithm 1 steps
1. Define impedance combinations
2. Define nodal equation
3. Loop through all impedance combinations and find H(s)

### 1. Define Z Combination

In [4]:
# (1) Define impedance combinations
def getCommonSourceImpedanceCombinations():
    # Define impedance arrays
    Zzs = [Rs,
           1/(s*C1),
           s*L1]

    Zz1 = [R1,
           1/(s*C1),
           R1/(1 + R1*C1*s),
           R1 + 1/(C1*s),
           s*L1 + 1/(s*C1)]

    Zz2 = [R2,                  # R
           1/(s*C2),            # C
           R2/(1 + R2*C2*s),    # R || C
           R2 + 1/(C2*s),       # R + C
           s*L2 + 1/(s*C2)]     # L + C

    Zz3 = [R3,
           1/(s*C3),
           s*L3,
           R3/(1 + R3*C3*s),
           (s*L3 + 1/(s*C3)),
           (L3*s)/(1 + L3*C3*s**2)]

    Zz4 = [R4,
           1/(s*C4),
           s*L4,
           R4/(1 + R4*C4*s),
           (s*L4 + 1/(s*C4)),
           (L4*s)/(1 + L4*C4*s**2)]

    Zz5 = [R5,
           1/(s*C5),
           s*L5,
           R5/(1 + R5*C5*s),
           (s*L5 + 1/(s*C5)),
           (L5*s)/(1 + L5*C5*s**2)]

    ZzL = [RL,
           1/(s*CL),
           s*LL,
           RL/(1 + RL*CL*s),
           (LL*s)/(1 + LL*CL*s**2)]

    # Combine all impedances
    impedance_combinations = []
    for zs in Zzs:
        for z1 in Zz1:
            for z2 in Zz2:
                for z3 in Zz3:
                    for z4 in Zz4:
                        for z5 in Zz5:
                          for zL in ZzL:
                            impedance_combinations.append((zs, z1, z2, z3, z4, z5, zL))

    keys = ["Zs", "Z1", "Z2", "Z3", "Z4", "Z5", "ZL"]
    return impedance_combinations


def getCommonGateImpedanceCombinations():
    # Define impedance arrays (Based on Table 1)
    Zz1 = [R1]                      # R

    Zz2 = [R2,                                            # R
           1/(s*C2),                                      # C
           R2/(1 + R2*C2*s),                              # R || C
           R2 + 1/(C2*s),                                 # R + C
           s*L2 + 1/(s*C2),                               # L + C
           R2 + s*L2 + 1/(s*C2),                          # R + L + C
           R2 + (s*L2/(1 + L2*C2*s**2)),                  # R + (L || C)
           R2*(s*L2 + 1/(s*C2))/(R2 + (s*L2 + 1/(s*C2)))  # R2 || (L2 + C2)
           ]

    Zz3 = [R3,                                            # R
           s*L3,                                          # L
           1/(s*C3),                                      # C
           R3/(1 + R3*C3*s),                              # R || C
           R3 + 1/(C3*s),                                 # R + C
           (s*L3 + 1/(s*C3)),                             # L + C
           (L3*s)/(1 + L3*C3*s**2),                       # L || C
           R3 + s*L3 + 1/(s*C3),                          # R + L + C
           (1/R3 + s*C3+ 1/(s*L3))**-1,                   # R || L || C
           R3 + (s*L3/(1 + L3*C3*s**2)),                  # R + (L || C)
           R3*(s*L3 + 1/(s*C3))/(R3 + (s*L3 + 1/(s*C3)))  # R || (L + C)
           ]


    Zz4 = [R4,                                            # R
           s*L4,                                          # L
           1/(s*C4),                                      # C
           R4/(1 + R4*C4*s),                              # R || C
           R4 + 1/(C4*s),                                 # R + C
           (s*L4 + 1/(s*C4)),                             # L + C
           (L4*s)/(1 + L4*C4*s**2),                       # L || C
           R4 + s*L4 + 1/(s*C4),                          # R + L + C
           (1/R4 + s*C4+ 1/(s*L4))**-1,                   # R || L || C
           R4 + (s*L4/(1 + L4*C4*s**2)),                  # R + (L || C)
           R4*(s*L4 + 1/(s*C4))/(R4 + (s*L4 + 1/(s*C4)))  # R || (L + C)
           ]

    Zz5 = [R4,                                            # R
           s*L4,                                          # L
           1/(s*C4),                                      # C
           R4/(1 + R4*C4*s),                              # R || C
           R4 + 1/(C4*s),                                 # R + C
           (s*L4 + 1/(s*C4)),                             # L + C
           (L4*s)/(1 + L4*C4*s**2),                       # L || C
           R4 + s*L4 + 1/(s*C4),                          # R + L + C
           (1/R4 + s*C4+ 1/(s*L4))**-1,                   # R || L || C
           R4 + (s*L4/(1 + L4*C4*s**2)),                  # R + (L || C)
           R4*(s*L4 + 1/(s*C4))/(R4 + (s*L4 + 1/(s*C4)))  # R || (L + C)
           ]

    ZzL = [RL,                                            # R
           s*LL,                                          # L
           1/(s*CL),                                      # C
           RL/(1 + RL*CL*s),                              # R || C
           RL + 1/(CL*s),                                 # R + C
           (s*LL + 1/(s*CL)),                             # L + C
           (LL*s)/(1 + LL*CL*s**2),                       # L || C
           RL + s*LL + 1/(s*CL),                          # R + L + C
           (1/RL + s*CL+ 1/(s*LL))**-1,                   # R || L || C
           RL + (s*LL/(1 + LL*CL*s**2)),                  # R + (L || C)
           RL*(s*LL + 1/(s*CL))/(RL + (s*LL + 1/(s*CL)))  # R || (L + C)
           ]

    # Combine all impedances
    impedance_combinations = []

    for z1 in Zz1:
        for z2 in Zz2:
            for z3 in Zz3:
                for z4 in Zz4:
                    for z5 in Zz5:
                        for zL in ZzL:
                            impedance_combinations.append((z1, z2, z3, z4, z5, zL))

    keys = ["Z1", "Z2", "Z3", "Z4", "Z5", "ZL"]
    return impedance_combinations

len(getCommonSourceImpedanceCombinations()[0]), len(getCommonGateImpedanceCombinations()[0])

(7, 6)

### 2. Define Nodal Equations

In [5]:
# (2) Define Nodal Equations
def getCommonSourceEquation(TransmissionMatrices, Z_arr = symbols('Zs Z1 Z2 Z3 Z4 Z5 ZL')):
    # Get symbolic variables (CS)
    Zs, Z1 , Z2 , Z3 , Z4 , Z5 , ZL = Z_arr
    I1a, I1b, I2a, I2b = symbols('I1a I1b I2a I2b')
    Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')
    T_a, T_b = TransmissionMatrices

    # Define nodal equations (Eq. 3a-3i) -> list[ Eq(left-hand-side, right-hand-side), ... ]
    equations = [
        # 3a
        Eq(0, (I1a + I2a + I1b + I2b + (Vb-Vx)/Z1 + (Von - Vx)/Z2 + (Vop - Vx)/Z2 + (Va - Vx)/Z1)),
        # 3b
        Eq(I1a, ((Vip - Va)/Zs + (Von - Va)/Z3 + (Vx - Va)/Z1 + (Vop - Va)/Z5)),
        # 3c
        Eq(I1b, (Vin - Vb)/Zs + (Vop - Vb)/Z3 + (Vx - Vb)/Z1 + (Von - Vb)/Z5),
        # 3d
        Eq(I2a, (Va - Von)/Z3 + (Vx - Von)/Z2 + (0 - Von)/ZL + (Vop - Von)/Z4 + (Vb - Von)/Z5),
        # 3e
        Eq(I2b, (Vb - Vop)/Z3 + (Vx - Vop)/Z2 + (0 - Vop)/ZL + (Von - Vop)/Z4 + (Va - Vop)/Z5),
        # 3f
        Eq(Vx + V2a, Vop),
        Eq(Vx + V2b, Von),
        # 3g
        Eq(Vx + V1a, Va),
        Eq(Vx + V1b, Vb),
        # 3h: Transistor a
        Eq(V1a, T_a[0,0]*V2a - T_a[0,1]*I2a),
        Eq(I1a, T_a[1,0]*V2a + T_a[1,1]*I2a),
        # 3i: Transistor b
        Eq(V1b, T_b[0,0]*V2b - T_b[0,1]*I2b),
        Eq(I1b, T_b[1,0]*V2b + T_b[1,1]*I2b)
        ]

    return equations

def getCommonGateEquation(TransmissionMatrices, Z_arr = symbols('Z1 Z2 Z3 Z4 Z5 ZL')):
    # Get symbolic variables (CG)
    Z1 , Z2 , Z3 , Z4 , Z5 , ZL = Z_arr
    I1a, I1b, I2a, I2b, Iip, Iin = symbols('I1a I1b I2a I2b Iip Iin')
    Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')
    T_a, T_b = TransmissionMatrices

    # Define nodal equations (Eq. 4a-4h) -> list[ Eq(left-hand-side, right-hand-side), ... ]
    equations = [
         # 4a
         Eq(0, (Iip + I1a + I2a + (0 - Vip)/Z1 + (Vop - Vip)/Z2 + (Von - Vip)/Z5)),
         # 4b
         Eq(0, (Iin + I1b + I2b + (0 - Vin)/Z1 + (Von - Vin)/Z2 + (Vop - Vin)/Z5)),
         # 4c
         Eq(I2a, ((Vip - Vop)/Z2 + (Vx - Vop)/Z3 + (Von - Vop)/Z4 + (Vin - Vop)/Z5 + (0 - Vop)/ZL)),
         # 4d
         Eq(I2b, ((Vin - Von)/Z2 + (Vx - Von)/Z3 + (Vop - Von)/Z4 + (Vip - Von)/Z5 + (0 - Von)/ZL )),
         # unranked equation in the paper (between 4d and 4e)
         Eq(I1a, ((Vop - Vx)/Z3 + (Von - Vx)/Z3 - I1b)),
         # 4e
         Eq(Vop, Vip + V2a),
         Eq(Von, Vin + V2b),
         # 4f
         Eq(Vip, Vx - V1a),
         Eq(Vin, Vx - V1b),
         # 3g
         Eq(V1a, T_a[0,0]*V2a - T_a[0,1]*I2a),
         Eq(V1b, T_b[0,0]*V2b - T_b[0,1]*I2b),
         # 3h
         Eq(I1a, T_a[1,0]*V2a + T_a[1,1]*I2a),
         Eq(I1b, T_b[1,0]*V2b + T_b[1,1]*I2b)
     ]

    return equations

print(f"Common source equation count: {len(getCommonSourceEquation(TransmissionMatrices=[T_a, T_b]))}")
print(f"Common gate equation count: {len(getCommonGateEquation(TransmissionMatrices=[T_a, T_b]))}")

Common source equation count: 13
Common gate equation count: 13


### 3. Get Transfer Functions

In [6]:
# (3 - CS) Get transfer functions (TFs)
def _getCommonSourceBaseHs():
    print(f"====CommonSource====")
    TransmissioMatrices = [T_a, T_b]
    Iip, Iin, I1a, I1b, I2a, I2b  = symbols('Iip Iin I1a I1b I2a I2b')
    Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')

    # Define nodal equations
    equations = getCommonSourceEquation(TransmissioMatrices)
    print("(1) set up the nodal equation")

    # Solve for transfer function
    solution = solve(equations, [
        Vop, Von, I1a, I2a, V2a, V1a, V1b, V2b,
        I1b, I2b, Vip, Vin, Vx
    ])
    print("(2) solved the base transfer function")

    if solution:
        print("FOUND THE BASE TF")
        baseHs = (solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin])
        baseHs = simplify(baseHs.factor())
        return baseHs
    print("Could not find the base Hs")
    return None

def getCommonSourceTFs_V2(impedanceBatch, baseHs):

    TFs = []
    if baseHs:
        for zCombo in tqdm(impedanceBatch, desc="Getting the TFs (CS)", unit="combo"):
            Zs, Z1, Z2, Z3, Z4, Z5, ZL = zCombo

            sub_dict = {symbols("Zs") : Zs,
                        symbols("Z1") : Z1, 
                        symbols("Z2") : Z2, 
                        symbols("Z3") : Z3, 
                        symbols("Z4") : Z4, 
                        symbols("Z5") : Z5,
                        symbols("ZL") : ZL}
            
            Hs = baseHs.subs(sub_dict)
            Hs = simplify((Hs.factor()))
            TFs.append(Hs)

    return TFs

# _getCommonSourceBaseHs()

In [7]:
# (3 - CG) Get transfer functions (TFs)
def _getCommonGateBaseHs():
    print(f"====CommonGate====")
    TFs = []
    TransmissioMatrices = [T_a, T_b]

    # Get symbolic variables (CG)
    Iip, Iin, I1a, I1b, I2a, I2b = symbols('Iip Iin I1a I1b I2a I2b')
    Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')

    # Define nodal equations
    equations = getCommonGateEquation(TransmissioMatrices)
    print("(1) set up the nodal equation")

    # Solve for generic transfer function
    solution = solve(equations, [
        Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx,
        Iip, Iin, I1a, I1b, I2a, I2b
    ])
    print("(2) solved the base transfer function")

    if solution:
        print("FOUND THE BASE TF")
        baseHs = (solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin])
        baseHs = simplify((baseHs.factor()))
        return baseHs
    
    return None


def getCommonGateTFs_V2(impedanceBatch, baseHs):

    TFs = []
    if baseHs:
        for zCombo in tqdm(impedanceBatch, desc="Getting the TFs (CG)", unit="combo"):
            Z1, Z2, Z3, Z4, Z5, ZL = zCombo

            sub_dict = {symbols("Z1") : Z1, 
                        symbols("Z2") : Z2, 
                        symbols("Z3") : Z3, 
                        symbols("Z4") : Z4, 
                        symbols("Z5") : Z5,
                        symbols("ZL") : ZL}
            
            Hs = baseHs.subs(sub_dict)
            Hs = simplify((Hs.factor()))
            TFs.append(Hs)

    return TFs

# _getCommonGateBaseHs()

## Algorithm 2: Computing w0, Q, Qzz, K_LP, K_HP, K_BP
IN -> H(s), out -> temp = 0 if valid values (else temp = 1, meaning all values are invalid since filter total order is out of bound)

In [8]:
def _computeFilterParameters(tf): # HELPER FUNCTION
    s = symbols('s')
    temp = 0

    # Determine numerator and denominator of H(s)
    dn = denom(tf)  # Denominator
    nm = numer(tf)  # Numerator
    DenOrder = degree(dn, s)
    NumOrder = degree(nm, s)

    # Coefficients of denominator
    a = dn.coeff(s, 2)  # Coefficient of s^2
    b = dn.coeff(s, 1)  # Coefficient of s^1
    c = dn.coeff(s, 0)  # Coefficient of s^0

    # Validate filter order and coefficients
    if a == 0 or b == 0 or c == 0 or DenOrder > 2 or NumOrder > DenOrder:
        temp = 1
        return temp, {'message': "Invalid "}

    # Compute Q, w, Bandwidth
    Q = simplify((a / b) * sqrt(c / a))
    wo_sqr = simplify(c / a)
    wo = sqrt(wo_sqr)
    Bandwidth = wo / Q

    # Coefficients of numerator
    bhp = nm.coeff(s, 2)  # Coefficient of s^2
    bbp = nm.coeff(s, 1)  # Coefficient of s^1
    blp = nm.coeff(s, 0)  # Coefficient of s^0

    # Calculate filter constants
    K_LP = simplify(blp / (a * wo**2))
    K_HP = simplify(bhp / a)
    K_BP = simplify(bbp / (a * Bandwidth))

    # Compute Qz if conditions are met
    Qz = None
    if K_LP == K_HP and K_BP != 0 and K_LP != 0:
        Qz = simplify(K_LP * Q / K_BP)

    # Return results
    return temp, {
        "Q": Q,
        "wo": wo,
        "Bandwidth": Bandwidth,
        "K_LP": K_LP,
        "K_HP": K_HP,
        "K_BP": K_BP,
        "Qz": Qz,
    }

def computeFilterParameters(TFs, impedanceCombinations):
    """
    Compute filter parameters for a list of transfer functions and impedance combinations.

    Parameters:
    - TFs: List of transfer functions.
    - impedanceCombinations: List of corresponding impedance combinations.

    Returns:
    - List of dictionaries containing impedance combination, transfer function, and parameters.
    """
    output = []
    count = 0
    # Wrap the zip iterator with tqdm for progress tracking
    for tf, impedanceCombo in tqdm(zip(TFs, impedanceCombinations),
                                    total=len(TFs),
                                    desc="Computing filter parameters",
                                    unit="filter"):
        temp, parameters = _computeFilterParameters(tf)
        if temp == 0:
            output.append({"Impedance": impedanceCombo, "TF": tf, "Parameters": parameters})
            count += 1
        else:
            output.append({"Impedance": impedanceCombo, "TF": tf, "Parameters": None})
    return output

# # Test Compute filter parameters
# s = symbols("s")
# tf = (s**2 + 2*s + 1)/(s**2 + 4*s + 4)

# temp, parameters = _computeFilterParameters(tf)
# pprint(parameters)

# tfs = [(s**2 - 1)/(3*s**2 + 4*s + 4), (2*s + 1)/(s**2 + 1)]
# output, count = computeFilterParameters(tfs, [1, 2])
# print()
# print("++++++++")
# pprint(output)

## Algorithm 3: Determine Filter Type
* IN -> Parameters from Algorithm 2: w0, Q, Qz, K_LP, K_HP, K_BP
* OUT -> Filter type classification (e.g., HP, BP, LP, BS, GE)





In [9]:
def getFilterType(parameters):
    """
    Determine the type of filter based on K_LP, K_HP, K_BP, and Qz.

    Parameters:
    - parameters: dict containing Q, wo, K_LP, K_HP, K_BP, and Qz.

    Returns:
    - List of filter types and associated parameters.
    """
    temp = 0  # Use as a validity flag
    K_LP = parameters.get("K_LP", 0)
    K_HP = parameters.get("K_HP", 0)
    K_BP = parameters.get("K_BP", 0)
    Q = parameters.get("Q")
    wo = parameters.get("wo")
    Qz = parameters.get("Qz")

    # Result containers
    filter_results = {
        "HP": None,
        "BP": None,
        "LP": None,
        "BS": None,
        "GE": None
    }

    # Classification based on filter constants
    if temp == 0 and K_BP == 0 and K_LP == 0:
        filter_results["HP"] = {"wo": wo, "Q": Q, "K_HP": K_HP}

    if temp == 0 and K_HP == 0 and K_LP == 0:
        filter_results["BP"] = {"wo": wo, "Q": Q, "K_BP": K_BP}

    if temp == 0 and K_HP == 0 and K_BP == 0:
        filter_results["LP"] = {"wo": wo, "Q": Q, "K_LP": K_LP} 

    if temp == 0 and K_BP == 0 and K_HP != 0 and K_LP != 0:
        filter_results["BS"] = {"wo": wo, "Q": Q, "K_HP": K_HP, "K_LP": K_LP}

    if temp == 0 and K_LP == K_HP and K_BP != 0 and K_LP != 0:
        if Qz is None:
            Qz = simplify(K_LP * Q / K_BP)
        filter_results["GE"] = {"wo": wo, "Q": Q, "Qz": Qz, "K_LP": K_LP}

    return filter_results

def classifyFilterParameters(filterParameter_results):
    """
    Classify transfer functions into filter types.

    Parameters:
    - tf_results: List of transfer function results from `computeFilterParameters`.

    Returns:
    - Filter classifications.
    """
    classifiedResults = []
    count = 0
    for result in filterParameter_results:
        parameters = result.get("Parameters")
        if parameters:
            filter_types = getFilterType(parameters)
            classifiedResults.append({
                "Impedance": result["Impedance"],
                "TF": result["TF"],
                "Filter Types": filter_types
            })
            count += 1
        else:
            classifiedResults.append({
                "Impedance": result["Impedance"],
                "TF": result["TF"],
                "Filter Types": None
            })
    return classifiedResults

# # Test Compute filter parameters
# s = symbols("s")
# tf = (s**2 + 2*s + 1)/(s**2 + 4*s + 4)

# temp, parameters = _computeFilterParameters(tf)
# pprint(parameters)

# tfs = [(s**2 - 1)/(3*s**2 + 4*s + 4), (2*s + 1)/(s**2 + 1)]
# output, count = computeFilterParameters(tfs, [1, 2])
# print()
# print("++++++++")
# pprint(output)
# print("++++++++")

# classification, count = classifyFilters(output)
# pprint(classification)

## Helper Functions

### Filtering and Processing

In [10]:
def findFilterInClassification(filterType, classifications):

    output = []

    for entity in classifications:
        if (entity["Filter Types"]) and (entity["Filter Types"][filterType]):
            output.append({
                           "Impedance" : entity["Impedance"],
                           "TF": entity["TF"],
                           "Parameters" : entity["Filter Types"][filterType]
                           })
            
    if len(output) == 0:
        print(f"No filter of type {filterType} was found")
    return output



### Printing

In [11]:
def save_TFs_as_pdf(TFs, output_filename):
    """
    Saves a list of transfer functions as rendered mathematical expressions to a PDF.

    Parameters:
    - TFs: List of Sympy expressions representing transfer functions.
    - output_filename: Name of the output PDF file.
    """
    output_filename = "Outputs/" + output_filename + ".pdf"
    os.makedirs("Outputs", exist_ok=True)

    # Prepare the PDF canvas
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter
    margin = 50  # Margin around the page
    y_position = height - margin  # Starting position for writing

    for i, tf in enumerate(TFs, 1):
        # Render the transfer function as an image
        image_filename = f"Outputs/TF_{i}.png"
        tf_latex = f"H(s) = {latex(tf)}"
        sympy.preview(
            tf_latex,
            output='png',
            viewer='file',
            filename=image_filename,
            euler=False,  # Use standard fonts instead of Euler fonts
        )

        # Check if there's enough space for the next image; if not, start a new page
        image_height = 100  # Adjust if needed for image size
        if y_position - image_height < margin:
            c.showPage()  # Add a new page
            y_position = height - margin

        # Draw the image on the PDF
        c.drawImage(image_filename, margin, y_position - image_height, width=width - 2 * margin, height=image_height)

        # Move the y_position for the next item
        y_position -= image_height + 20  # Add spacing between entries

    c.save()
    print(f"PDF saved to {output_filename}")

    # Cleanup: Remove temporary image files
    for i in range(1, len(TFs) + 1):
        os.remove(f"Outputs/TF_{i}.png")


def save_TFs_as_LaTex(TFs, output_filename):
    """ 
        # !sudo apt update
        # !sudo apt install texlive-latex-base

    # # Compile the LaTeX file into a PDF (make sure pdflatex is installed)
    # !pdflatex output_filename.tex
    
    # Optional: download the PDF
    # files.download("transfer_functions.pdf")

    # Create a Latex file to report the transfer functions
    """

    header = r"""
    \documentclass{article}
    \usepackage{amsmath}
    \usepackage{geometry}
    \geometry{landscape, a1paper, margin=1in}  % Adjust paper size and margins
    \begin{document}
    \section*{Derived Transfer Functions}
    """

    footer = r"\end{document}"

    # LaTeX filename
    output_filename = "Outputs/" + output_filename + ".tex"
    os.makedirs("Outputs", exist_ok=True)

    # Write the LaTeX code into the file
    with open(output_filename, "w") as latex_file:
        latex_file.write(header)
        for i, tf in enumerate(TFs, 1):
            latex_file.write(f"\\subsection*{{Transfer Function {i}}}\n")
            latex_file.write(f"\\[ H(s) = {latex(tf)} \\]\n")
            # # Add a page break after each equation (optional)
            # latex_file.write("\\newpage\n")
        latex_file.write(footer)


# Testing all algorithms

In [ ]:
# Testing Algorithm 1
impedance_combinations = getCommonSourceImpedanceCombinations()
print("Number of impedance combinations: {}".format(len(impedance_combinations)))

impedanceBatch = impedance_combinations[0:10]
TFs = getCommonSourceTFs(impedanceBatch)

# Output summary of results
print("================")
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("TF {}: {}".format(i, tf))


In [ ]:
# Testting Algorithm 2
filterParameters = computeFilterParameters(TFs, impedanceBatch)
outputs = classifyFilterParameters(filterParameters)

In [ ]:
# Example usage
s = symbols("s")
TFs = [
    simplify((s + 1) / (s**2 + 3 * s + 2)),
    simplify(1 / (s**2 + s + 1))
]
save_TFs_as_pdf(TFs, "test.pdf")

parameters = computeFilterParameters(TFs, [1, 2])
print()
print("++++++++")

output = classifyFilterParameters(parameters)
pprint(output)

print("++++++++")
x = findFilterInClassification("LP", output)
pprint(x)

# Experiment Runs

In [12]:
"""
Select from the below defintions
T_select ={
"simple" : Matrix([[0, -1/gm],[0, 0]]),
"symbolic" : Matrix([[a11, a12],[a21, a22]]),
"some_parasitic" : Matrix([[-1/(gm*ro), -1/gm],[0, 0]]),
"full_parasitic" : Matrix([[(1/ro + s*Cgd)/(s*Cgd - gm), 1/(s*Cgd - gm)],[(Cgd*Cgs*ro*s + Cgd*gm*ro + Cgs + Cgd)*s/(s*Cgd - gm), (Cgs+Cgd)*s/(s*Cgd - gm)]])
}
"""

# Select a transimision matrix
T_type = "symbolic"
T_a = T_select[T_type]
T_b = T_select[T_type]


## Common Source

In [13]:
# Improves efficiency by pre-computing the Transfer Function H(s, Z)
baseHs_CS = _getCommonSourceBaseHs() 

====CommonSource====
(1) set up the nodal equation
(2) solved the base transfer function
FOUND THE BASE TF


In [ ]:
# Sanity Check 
impedance_combinations = getCommonSourceImpedanceCombinations()
batchSize = 10
batch = impedance_combinations[0:batchSize]

TFs = getCommonSourceTFs_V2(batch, baseHs_CS)

# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("TF {}: {}".format(i, tf))

# save_TFs_as_LaTex(TFs, "CS_testRun")
# !latex Outputs/CS_testRun.tex

Getting the TFs (CS): 100%|██████████| 10/10 [00:02<00:00,  3.75combo/s]

Number of transfer functions found: 10
TF 1: R1*R2*R4*RL*(-R3*R5 + R3*a12 - R5*a12)/(-R1*R2*R3*R4*R5*RL*Rs*a21 - R1*R2*R3*R4*R5*RL*a11 - R1*R2*R3*R4*R5*Rs*a22 + R1*R2*R3*R4*R5*a12 + R1*R2*R3*R4*RL*Rs*a11*a22 - R1*R2*R3*R4*RL*Rs*a11 + R1*R2*R3*R4*RL*Rs*a12*a21 - R1*R2*R3*R4*RL*Rs*a22 + R1*R2*R3*R4*RL*Rs + R1*R2*R3*R4*RL*a12 + R1*R2*R3*R4*Rs*a12 - 2*R1*R2*R3*R5*RL*Rs*a22 + 2*R1*R2*R3*R5*RL*a12 + 2*R1*R2*R3*RL*Rs*a12 - R1*R2*R4*R5*RL*Rs*a11*a22 - R1*R2*R4*R5*RL*Rs*a11 - R1*R2*R4*R5*RL*Rs*a12*a21 - R1*R2*R4*R5*RL*Rs*a22 - R1*R2*R4*R5*RL*Rs + R1*R2*R4*R5*RL*a12 + R1*R2*R4*R5*Rs*a12 + 4*R1*R2*R4*RL*Rs*a12 + 2*R1*R2*R5*RL*Rs*a12 - R1*R3*R4*R5*RL*Rs*a22 + R1*R3*R4*R5*RL*a12 + R1*R3*R4*RL*Rs*a12 + R1*R4*R5*RL*Rs*a12 - R2*R3*R4*R5*RL*Rs*a11 + R2*R3*R4*R5*Rs*a12 + R2*R3*R4*RL*Rs*a12 + 2*R2*R3*R5*RL*Rs*a12 + R2*R4*R5*RL*Rs*a12 + R3*R4*R5*RL*Rs*a12)
TF 2: R1*R2*R4*(-R3*R5 + R3*a12 - R5*a12)/(-CL*R1*R2*R3*R4*R5*Rs*a22*s + CL*R1*R2*R3*R4*R5*a12*s + CL*R1*R2*R3*R4*Rs*a12*s + CL*R1*R2*R4*R5*Rs*a12*s + 

In [ ]:
# !latex Outputs/CS_testRun.tex # uncomment to compile the CS_test.tex

## Common Gate

In [13]:
baseHs_CG = _getCommonGateBaseHs()
baseHs_CG

====CommonGate====
(1) set up the nodal equation
(2) solved the base transfer function
FOUND THE BASE TF


Z3*Z4*ZL*(Z2*Z5*a11 - Z2*Z5 - Z2*a12 + Z5*a12)/(Z2*Z3*Z4*Z5*ZL*a11 + Z2*Z3*Z4*Z5*a12 + Z2*Z3*Z4*ZL*a12 + 2*Z2*Z3*Z5*ZL*a12 + Z2*Z4*Z5*ZL*a12 + Z3*Z4*Z5*ZL*a12)

In [14]:
impedance_combinations = getCommonGateImpedanceCombinations()
batch = impedance_combinations[0:10]
# batch = [[1, 1, 1, 1, 1, 1]]

TFs = getCommonGateTFs_V2(batch, baseHs_CG)

# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("H(s) {}: {}".format(i, tf))

# save_TFs_as_LaTex(TFs, "CG_testRun")

Getting the TFs (CG): 100%|██████████| 10/10 [00:00<00:00, 13.13combo/s]

Number of transfer functions found: 10
H(s) 1: R3*RL*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(R2*R3*R4*RL*a11 + R2*R3*R4*a12 + 3*R2*R3*RL*a12 + R2*R4*RL*a12 + R3*R4*RL*a12)
H(s) 2: LL*R3*s*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(LL*R2*R3*R4*a11*s + 3*LL*R2*R3*a12*s + LL*R2*R4*a12*s + LL*R3*R4*a12*s + R2*R3*R4*a12)
H(s) 3: R3*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(CL*R2*R3*R4*a12*s + R2*R3*R4*a11 + 3*R2*R3*a12 + R2*R4*a12 + R3*R4*a12)
H(s) 4: R3*RL*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(CL*R2*R3*R4*RL*a12*s + R2*R3*R4*RL*a11 + R2*R3*R4*a12 + 3*R2*R3*RL*a12 + R2*R4*RL*a12 + R3*R4*RL*a12)
H(s) 5: R3*(CL*RL*s + 1)*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(CL*R2*R3*R4*RL*a11*s + CL*R2*R3*R4*a12*s + 3*CL*R2*R3*RL*a12*s + CL*R2*R4*RL*a12*s + CL*R3*R4*RL*a12*s + R2*R3*R4*a11 + 3*R2*R3*a12 + R2*R4*a12 + R3*R4*a12)
H(s) 6: R3*(CL*LL*s**2 + 1)*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(CL*LL*R2*R3*R4*a11*s**2 + 3*CL*LL*R2*R3*a12*s**2 + CL*LL*R2*R4*a12*s**2 + CL*LL*R3*R4*a12*s**2 + CL*R2*R3*R4*a12*s + R2*R3

In [17]:
# Preview the TF
TFs[0]

R3*RL*(R2*R4*a11 - R2*R4 - R2*a12 + R4*a12)/(R2*R3*R4*RL*a11 + R2*R3*R4*a12 + 3*R2*R3*RL*a12 + R2*R4*RL*a12 + R3*R4*RL*a12)

In [ ]:
!pdflatex Outputs/CS_testRun.tex

# Saving the environment parameters
Use the code block below to save the experiment.

**Requires the Dill library to be pip installed**

In [ ]:
# Saving the variable with dill
variables = {'TFs': TFs, 'Impedance_combinations': impedance_combinations}

with open('sympy_variables.dill', 'wb') as f:
    dill.dump(variables, f)

In [ ]:
# Loading the variabels with dill
with open('sympy_variables.dill', 'rb') as f:
    loaded_variables = dill.load(f)

# Access the loaded symbolic expressions and variables
TFs = loaded_variables['TFs']
Impedance_combinations = loaded_variables['Impedance_combinations']

# Archive

## OLD approach (Common Source)

In [ ]:
def getTFs(batch):
  numOfSolutions = 0
  counter = 0
  TFs = []
  for combo in tqdm(batch, desc="Getting the TFs", unit="combo"):
      Zs, Z1, Z2, Z3, Z4, Z5, ZL = combo

      Vip, Va, Von, Vx, Vop, Vb, Vin, I1a, I1b, I2a, I2b = symbols('Vip Va Von Vx Vop Vb Vin I1a I1b I2a I2b')
      V2a, V2b, V1a, V1b =  symbols('V2a V2b V1a V1b')

      # Define nodal equations (Eq. 3a-3i) -> list[ Eq(left-hand-side, right-hand-side), ... ]
      equations = [
          # 3a
          Eq(0, I1a + I2a + I1b + I2b + (Vb-Vx)/Z1 + (Von - Vx)/Z2 + (Vop - Vx)/Z2 + (Va - Vx)/Z1),
          # 3b
          Eq(I1a, (Vip - Va)/Zs + (Von - Va)/Z3 + (Vx - Va)/Z1 + (Vop - Va)/Z5),
          # 3c
          Eq(I1b, (Vin - Vb)/Zs + (Vop - Vb)/Z3 + (Vx - Vb)/Z1 + (Von - Vb)/Z5),
          # 3d
          Eq(I2a, (Va - Von)/Z3 + (Vx - Von)/Z2 + (0 - Von)/ZL + (Vop - Von)/Z4 + (Vb - Von)/Z5),
          # 3e
          Eq(I2b, (Vb - Vop)/Z3 + (Vx - Vop)/Z2 + (0 - Vop)/ZL + (Von - Vop)/Z4 + (Va - Vop)/Z5),
          # 3f
          Eq(Vx + V2a, Vop),
          Eq(Vx + V2b, Von),
          # 3g
          Eq(Vx + V1a, Va),
          Eq(Vx + V1b, Vb),
          # 3h: Transistor a
          Eq(V1a, T_a[0,0]*V2a - T_a[0,1]*I2a),
          Eq(I1a, T_a[1,0]*V2a + T_a[1,1]*I2a),
          # 3i: Transistor b
          Eq(V1b, T_b[0,0]*V2b - T_b[0,1]*I2b),
          Eq(I1b, T_b[1,0]*V2b + T_b[1,1]*I2b)
          ]

      # Solve for transfer function
      solution = solve(equations, [
          Vop, Von, I1a, I2a, V2a, V1a, V1b, V2b,
          I1b, I2b, Vip, Vin, Vx
      ])

      if solution:
          # Compute transfer function and append to list
          numOfSolutions += 1
          Hs = (solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin])
          TFs.append(simplify(Hs))

      counter += 1
      if counter % 25 == 0:
          print("")
          print("Processed {} combinations".format(counter))
          print("Number of solutions found: {}".format(numOfSolutions))

  return TFs

In [ ]:
batch = [[Zs, Z1, Z2, Z3, Z4, Z5, ZL]]

TFs = getTFs(batch)

filterParameters = computeFilterParameters(TFs, batch)
classifyFilters(filterParameters)

# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("TF {}: {}".format(i, tf))

saveTFsLaTex("CS_testRun.tex", TFs)

In [ ]:
def getCommonSourceTFs(batch):
    print(f"====CommonSource==== batch size: {len(batch)}")
    TFs = []
    TransmissioMatrices = [T_a, T_b]
    Iip, Iin, I1a, I1b, I2a, I2b  = symbols('Iip Iin I1a I1b I2a I2b')
    Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')

    # Define nodal equations
    equations = getCommonSourceEquation(TransmissioMatrices)

    # Solve for transfer function
    solution = solve(equations, [
        Vop, Von, I1a, I2a, V2a, V1a, V1b, V2b,
        I1b, I2b, Vip, Vin, Vx
    ])

    if solution:
        baseHs = (solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin])
        baseHs = simplify(baseHs.factor())

    for zCombo in tqdm(batch, desc="Getting the TFs (CS)", unit="combo"):
        Zs, Z1, Z2, Z3, Z4, Z5, ZL    = zCombo

        sub_dict = {symbols("Zs") : Zs,
                    symbols("Z1") : Z1, 
                    symbols("Z2") : Z2, 
                    symbols("Z3") : Z3, 
                    symbols("Z4") : Z4, 
                    symbols("Z5") : Z5,
                    symbols("ZL") : ZL}
        
        Hs = baseHs.subs(sub_dict)
        Hs = simplify((Hs.factor()))
        TFs.append(Hs)

    return TFs

## OLD approach (Common Gate)

In [ ]:
# (3) Get transfer functions (TFs)
def getCommonGateTFs(batch):
    print(f"====CommonGate==== batch size: {len(batch)}")
    TFs = []
    TransmissioMatrices = [T_a, T_b]

    # Get symbolic variables (CG)
    Iip, Iin, I1a, I1b, I2a, I2b = symbols('Iip Iin I1a I1b I2a I2b')
    Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx = symbols('Vin V2a V2b V1a V1b Va Vb Von Vop Vip Vx')

    # Define nodal equations
    equations = getCommonGateEquation(TransmissioMatrices)
    print("(1) set up the nodal equation")

    # Solve for generic transfer function
    solution = solve(equations, [
        Vin, V2a, V2b, V1a, V1b, Va, Vb, Von, Vop, Vip, Vx,
        Iip, Iin, I1a, I1b, I2a, I2b
    ])
    print("(2) solved the base transfer function")

    if solution:
        print("FOUND A BASE TF")
        baseHs = (solution[Vop] - solution[Von]) / (solution[Vip] - solution[Vin])
        baseHs = simplify((baseHs.factor()))
    else:
        return

    count = 1
    for zCombo in tqdm(batch, desc="Getting the TFs (CG)", unit="combo"):
        # print(f"count {count} baseHs: {baseHs} zCombo: {zCombo}")
        count += 1
        Z1 , Z2 , Z3 , Z4 , Z5 , ZL = zCombo
        
        sub_dict = {symbols("Z1") : Z1, 
                    symbols("Z2") : Z2, 
                    symbols("Z3") : Z3, 
                    symbols("Z4") : Z4, 
                    symbols("Z5") : Z5,
                    symbols("ZL") : ZL}
        
        Hs = baseHs.subs(sub_dict)
        Hs = simplify((Hs.factor()))
        TFs.append(Hs)

    return baseHs, TFs

In [ ]:
# Sanity Check
impedance_combinations = getCommonGateImpedanceCombinations()
batch = impedance_combinations[0:10]
# batch = [[Z1, Z2, Z3, Z4, Z5, ZL]]
# batch = [[1, 1, 1, 1, 1, 1]]

TFs = getCommonGateTFs(batch)
# Output summary of results
print("Number of transfer functions found: {}".format(len(TFs)))
for i, tf in enumerate(TFs, 1):
    print("H(s) {}: {}".format(i, tf))


# filterParameters = computeFilterParameters(TFs, batch)
# classifyFilterParameters(filterParameters)



save_TFs_as_LaTex(TFs, "CG_testRun")

## Others

In [ ]:
import os
import matplotlib.pyplot as plt
from sympy import latex
from sympy import symbols
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

def save_TFs_as_pdf(TFs, output_filename):
    """
    Saves a list of transfer functions as rendered mathematical expressions to a PDF using Matplotlib.

    Parameters:
    - TFs: List of Sympy expressions representing transfer functions.
    - output_filename: Name of the output PDF file.
    """
    output_filename = "Outputs/" + output_filename + ".pdf"
    os.makedirs("Outputs", exist_ok=True)

    # Prepare the PDF canvas
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfgen import canvas
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter
    margin = 50  # Margin around the page
    y_position = height - margin  # Starting position for writing

    # Font settings for text
    font_name = "Helvetica"
    font_size = 12
    c.setFont(font_name, font_size)

    for i, tf in enumerate(TFs, 1):
        # Render the transfer function as LaTeX
        tf_latex = f"H(s) = {latex(tf)}"

        # Create a Matplotlib figure for rendering the LaTeX expression as an image
        fig, ax = plt.subplots(figsize=(8, 1))  # Adjust figure size
        ax.text(0.5, 0.5, f"${tf_latex}$", fontsize=16, ha='center', va='center')
        ax.axis('off')

        # Save the rendered figure to an image file
        image_filename = f"Outputs/TF_{i}.png"
        fig.savefig(image_filename, dpi=300, bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)

        # Check if there's enough space for the next image; if not, start a new page
        image_height = 100  # Adjust as needed for image size
        if y_position - image_height < margin:
            c.showPage()  # Add a new page
            y_position = height - margin

        # Draw the image on the PDF
        c.drawImage(image_filename, margin, y_position - image_height, width=width - 2 * margin, height=image_height)

        # Update y_position for the next transfer function
        y_position -= image_height + 20  # Add spacing between entries

    c.save()
    print(f"PDF saved to {output_filename}")

    # Cleanup: Remove temporary image files
    for i in range(1, len(TFs) + 1):
        os.remove(f"Outputs/TF_{i}.png")

# Example usage:
if __name__ == "__main__":
    s = symbols('s')
    TFs = [
        (s + 1) / (s**2 + 3*s + 2),
        1 / (s**2 + s + 1),
        (s**2 + 2) / (s**3 + 4*s + 5),
        (s**4 + s**3 + s**2 + s + 1) / (s**5 + s**4 + s**3 + s**2 + s + 1)
    ]
    save_TFs_as_pdf(TFs, "transfer_functions")
